gemaakt voor LIWO door [David Haasnoot](d.haasnoot@hkv.nl)

In [1]:
import dotenv
from pathlib import Path

In [2]:
from update_local_LDO_custom import haal_token_op, export_uit_LDO_custom, get_layer_names_from_scenario, haal_scenarios_op


Zie `readme.md` voor uitgebreid stappen plan voor het aanmaken van een api key.

In het kort
- Login & ga naar https://ldo.overstromingsinformatie.nl/auth/
- Onder `V1`, `POST auth/v1/personalapikeys`, klik op `try it out` en vervang body met:
```json
{
  "scope": "*:readwrite",
  "name": "personalAPIkeyLIWOexport...",
  "expiry_date": "2029-12-31T23:59:59.037Z",
  "revoked": false
}
```
- Pas de datum en name aan waar nodig, klik op `Excecute` in het blauw. In de response body staat: 
```json
{
...
  "key": "xxxxxx.yyyyyyyyyyyyyyy",
...
}
```
- Bewaar die hele `'key'` in een bestand die `.env` heet, zie `example.env` voor het formaat.

In [ ]:
# haal de API key op uit de .env file
if dotenv.load_dotenv():
    environmental_variables = dotenv.dotenv_values()
    LDO_api_key = environmental_variables["LDO_api_key"]
    TENANT = int(environmental_variables.get("TENANT", 1))

# Of zet hier de API key handmatig in
# LDO_api_key = "abcd"
# TENANT : int  = 1 # 0, 1, 2 ...
headers = haal_token_op(LDO_api_key, tenant=TENANT)
current_dir = Path.cwd()

In [16]:
gebiedsnaam = [
    "BRS%20gebied%20Brabantse%20Delta",
    "BRS%20gebied%20Noord-Brabant%20Oost",
    "BRS%20gebied%20Overijsselse%20Vecht",
    "BRS%20gebied%20Rivierenland",
    "BRS%20gebied%20Vallei%20en%20Veluwe",
    "BRS%20gebied%20Friesland",
    "BRS%20gebied%20Groningen%20en%20NO-Drenthe",
    "BRS%20gebied%20Scheldestromen",
    "BRS%20gebied%20ARK-NZK",
]
naam_filter = ""
for naam in gebiedsnaam:
    naam_filter += f"&gebiedsnaam={naam}"

In [18]:
maximum = 10_000
geo_filter = "&bbox=1.9492904467636265%2C50.770023203519315%2C8.995291621720668%2C52.492957339148916"
extra_filer = geo_filter + naam_filter
beschikbare_scenario_ids_geo = haal_scenarios_op(maximum, headers, extra_filter=extra_filer)

In [19]:
len(beschikbare_scenario_ids_geo)

30

Conclusie: we nergeren de geo filter

In [20]:
maximum = 10_000
beschikbare_scenario_ids = haal_scenarios_op(maximum, headers, extra_filter=naam_filter)

In [23]:
print(len(beschikbare_scenario_ids))
for id in set(beschikbare_scenario_ids).difference(set(beschikbare_scenario_ids_geo)):
    print(f"https://ldo.overstromingsinformatie.nl/scenarios/{id}")

37
https://ldo.overstromingsinformatie.nl/scenarios/114656
https://ldo.overstromingsinformatie.nl/scenarios/114657
https://ldo.overstromingsinformatie.nl/scenarios/111312
https://ldo.overstromingsinformatie.nl/scenarios/111313
https://ldo.overstromingsinformatie.nl/scenarios/113715
https://ldo.overstromingsinformatie.nl/scenarios/113716
https://ldo.overstromingsinformatie.nl/scenarios/113815


Kies welke scenarios gedownload worden (indien al deel gedownload)

In [28]:
partial_download = False
if partial_download:     
    downloaded_id_files = (current_dir / "downloaded_tiffs").glob("*")
    downloaded_ids = set([file.name.split("_")[-1] for file in downloaded_id_files])
    len(downloaded_ids)
    nieuwe_scenarios = set(downloaded_id_files).difference(beschikbare_scenario_ids)# [-2:]  # de laatste twee scenario's
    nieuwe_scenarios
else:
    nieuwe_scenarios = beschikbare_scenario_ids

In [29]:
len(nieuwe_scenarios)

37

In [30]:
df_layer_names = get_layer_names_from_scenario(nieuwe_scenarios, headers=headers)

Exporteert de gewenste bestanden (kan lang duren)

In [32]:
lst_zips_nieuwe_export = export_uit_LDO_custom(
                                        df_layer_names=df_layer_names,
                                        work_dir=current_dir,
                                        headers=headers,
)

0it [00:00, ?it/s]

37it [1:46:20, 172.43s/it]


In [33]:
from export_LDO import get_all_metadata
export_dir = current_dir / "downloaded_tiffs"

get_all_metadata(
            scenario_ids=df_layer_names.index.tolist(),
            fname=export_dir / "metadata.xlsx",
            headers=headers,
        )

In [35]:
df_layer_names.to_excel(export_dir / "layer_names.xlsx")